In [ ]:
import tkinter

from tkinter import *

import matplotlib.pyplot as plt
#Store the coordinates
coordinates = []

#Store the solution set
solution = []


#Left click on the Tkinter window to 
#Highlight the point and subsequently
#Store the coordinates of the point
def create_dot(event):
    x = event.x
    y = event.y
    r = 5
    my_canvas.create_oval(x+r,y+r,x-r,y-r,fill='black')
    coordinates.append((x,y))


#Reset using mouse wheel button
def clear(event):
    my_canvas.delete('all')
    coordinates.clear()
    solution.clear()


#Utility function to get which side of the
#line AB the coordinate in question lies on
def get_coordinate_side(A,B,coordinate):
    x1, y1 = A
    x2, y2 = B
    #ax + by + c
    x, y = coordinate
    a = y2 - y1
    b = x1 - x2
    c = x2*y1 - x1*y2
    f = a*x + b*y + c
    if f < 0:
        return 'right'
    elif f > 0:
        return 'left'
    else:
        return 'on-the-line'


def find_hull(partition,A,B):
    #Return if there are no points in the partition
    if len(partition) == 0:
        return
    else:
        #Get the Orthogonally farthest point C from line AB
        x1, y1 = A
        x2, y2 = B
        a = y2 - y1
        b = x1 - x2
        c = x2*y1 - x1*y2
        farthest_dist = -1
        C = None
        for coordinate in partition:
            x, y = coordinate
            f = abs(a*x + b*y + c)
            if f>farthest_dist:
                f = farthest_dist
                C = coordinate
        x,y = C
        
        #Update the solution set
        solution.remove((A,B))
        solution.append((A,C))
        solution.append((C,B))
        
        #Subdivide the solution
        partition.remove(C)
        ACright = []
        for coordinate in partition:
            coordinate_side = get_coordinate_side(A,C,coordinate)
            if coordinate_side == 'right':
                ACright.append(coordinate)
        CBright = []
        for coordinate in partition:
            coordinate_side = get_coordinate_side(C,B,coordinate)
            if coordinate_side == 'right':
                CBright.append(coordinate)
                
         
        #Divide and Conquer
        find_hull(ACright,A,C)
        find_hull(CBright,C,B)


def convex_hull(event):

    #Sort According to ascending order of X-coordinates
    #followed by ascending order of Y-coordinates (optional)
    sorted_coordinates = sorted(coordinates , key=lambda k: [k[0], k[1]])

    #Leftmost and Rightmost Coordinates A and B
    A = sorted_coordinates[0]
    B = sorted_coordinates[-1]

    #Partition Points on the left and right sides of line AB
    ABright = []
    BAright = []
    for coordinate in sorted_coordinates:
        coordinate_side = get_coordinate_side(A,B,coordinate)
        if coordinate_side == 'right':
            ABright.append(coordinate)
        elif coordinate_side == 'left':
            BAright.append(coordinate)
        else:
            pass

    #Append lines AB and BA to solution
    solution.append((A,B))
    solution.append((B,A))
    
    x1,y1 = A
    x2,y2 = B

    #Divide and Conquer
    find_hull(ABright,A,B)
    find_hull(BAright,B,A)

    #Draw Convex Hull for the final Solution
    for line in solution:
        (x1,y1),(x2,y2) = line
        my_canvas.create_line(x1,y1,x2,y2,width=2,fill='blue')


if __name__ == '__main__':
    my_window = tkinter.Tk()
    my_canvas = Canvas(my_window, width=500, height=500, background='white')
    #LMB to plant the points on the window
    my_canvas.bind('<Button-1>',create_dot)
    #RMB to get the convex hull
    my_canvas.bind('<Button-3>',convex_hull)
    #MWB to reset everything
    my_canvas.bind('<Button-2>',clear)
    my_canvas.grid(row=0,column=0)
    my_window.mainloop()